In [12]:
import numpy as np

MINM = -1000000000

#Hyperparameters
DELTA = 0.001
GAMMA = 0.999         
NOISE = 0  

#Define all pos
all_pos=[]
all_pos.append('W')
all_pos.append('N')
all_pos.append('E')
all_pos.append('S')
all_pos.append('C')

#Define all state
all_states=[]
all_states.append('D')
all_states.append('R')

max_mat = 2
max_arrow = 3

#Reward for each action
action_reward = {
    'UP': -10,
    'LEFT': -10,
    'DOWN': -10,
    'RIGHT': -10,
    'STAY': -10,
    'SHOOT': -10,
    'HIT': -10,
    'CRAFT': -10,
    'GATHER': -10,
    'NONE': -10
}

#Define rewards for all states
rewards = {}
for s in all_pos:
    rewards[s] = 0

#Dictionnary of possible actions. We have two "end" states (1,2 and 2,2)
actions = {
    'W':('RIGHT', 'STAY', 'SHOOT', 'NONE'), 
    'N':('DOWN', 'STAY', 'CRAFT', 'NONE'),
    'E':('LEFT', 'STAY', 'SHOOT', 'HIT', 'NONE'),
    'S':('UP', 'STAY', 'GATHER', 'NONE'),
    'C':('UP', 'DOWN', 'LEFT', 'RIGHT', 'STAY', 'SHOOT', 'HIT', 'NONE')
    }

#Define an initial policy
policy={}
for s in actions.keys():
    policy[s] = np.random.choice(actions[s])
print(policy)

#Define success probabilities for states
probability = {}
for s in all_pos:
    if s == 'E' or s == 'W':
        probability[s] = 1
    else:
        probability[s] = 0.85
print(probability)
        
#Define fail action for states
fail = {}
for s in all_pos:
    if s == 'E' or s == 'W':
        fail[s] = s
    else:
        fail[s] = 'E'
print(fail)

#Define initial value function 
V={}
for s in all_pos:
    if s in actions.keys():
        V[s] = 0
#     if s ==(2,2):
#         V[s]=-1
#     if s == (1,2):
#         V[s]=-1
#     if s == (2,3):
#         V[s]=1
print(V)

{'W': 'NONE', 'N': 'DOWN', 'E': 'NONE', 'S': 'GATHER', 'C': 'UP'}
{'W': 1, 'N': 0.85, 'E': 1, 'S': 0.85, 'C': 0.85}
{'W': 'W', 'N': 'E', 'E': 'E', 'S': 'E', 'C': 'E'}
{'W': 0, 'N': 0, 'E': 0, 'S': 0, 'C': 0}


In [19]:
def iterate():
    iteration = 0
    
    pos = 'W'
    mat = 0
    arrow = 0
    state = 'D'
    health = 4
    
    value = np.zeros((len(all_pos),max_mat,max_arrow,len(all_states),health))
    action = np.zeros((len(all_pos),max_mat,max_arrow,len(all_states),health))
    
    while True:
        biggest_change = 0
        print("iteration=",iteration)
        
        tmp = np.zeros((len(all_pos),max_mat,max_arrow,len(all_states),health))
        tmp[:] = MINM
        tmp[0,:,:] = 0
        
        for p in all_pos:
            for m in range(0,max_mat):
                for arr in range(0,max_arrow):
                    for s in all_states:
                        for h in range(1,health):
                            
#                             print("State: ",p,"  Value: ", V[s])           
#                             if p in policy:
#                             old_v = V[p]
#                             new_v = 0

                            # loop over actions
                            for a in actions[posn]:
                                if a == 'SHOOT' and arr == 0:
                                    continue
                                elif a == 'CRAFT' and (m == 0 or arr == 3):
                                    continue
                                elif a == 'GATHER' and m == 2:
                                    continue
                                    
                                state_value = 0
                                for p1 in all_pos:
                                    for m1 in range(0,max_mat):
                                        for arr1 in range(0,max_arrow):
                                            for s1 in all_states:
                                                for h1 in range(0,health):
                                                    
                                                    reward = action_reward[a]
                                                    if h1 == 0:
                                                        reward += 50
                                                    elif s == 'R' and s1 == 'D' and (p == 'E' or p == 'C'):
                                                        reward -= 40
                                if a == 'UP':
                                    if p == 'S':
                                        nxt = 'C'
                                    else:
                                        nxt = 'N'
                                elif a == 'DOWN':
                                    if p == 'N':
                                        nxt = 'C'
                                    else:
                                        nxt = 'S'
                                elif a == 'LEFT':
                                    if p == 'E':
                                        nxt = 'C'
                                    else:
                                        nxt = 'W'
                                elif a == 'RIGHT':
                                    if p == 'W':
                                        nxt = 'C'
                                    else:
                                        nxt = 'E'
                                elif a == 'STAY':
                                    nxt = p
                                else:
                                    nxt = p

                #                 if a == 'SHOOT':
                #                     nxt = [s[0]-1, s[1]]
                #                 if a == 'HIT':
                #                     nxt = [s[0]-1, s[1]]
                #                 if a == 'CRAFT':
                #                     nxt = [s[0]-1, s[1]]
                #                 if a == 'GATHER':
                #                     nxt = [s[0]-1, s[1]]
                #                 if a == 'NONE':
                #                     nxt = [s[0]-1, s[1]]

                                #Choose a new random action to do (transition probability)
                                nxt_2 = fail[p]
                #                 print(probability[s])
                                print(nxt)
                                print(V[nxt])
                                print(V[nxt_2])
                                sigma = probability[s]*V[nxt] + (1-probability[s])*V[nxt_2]

                                #Calculate the value
                                v = rewards[p] + (GAMMA * sigma) 
                                if v > new_v: #Is this the best action so far? If so, keep it
                                    new_v = v
                                    policy[s] = a
                                print("(",p,end=",")
                                print(m,end=",")
                                print(arr,end=",")
                                print(s,end=",")
                                print(h,end="):")
                                print(a,end=",[")
                                print(V[p],end="]")
           #Save the best of all actions for the state                                 
                V[p] = new_v
                biggest_change = max(biggest_change, np.abs(old_v - V[p]))

       #See if the loop should stop now         
        if biggest_change < DELTA:
            break
        iteration += 1

    for s in all_pos: 
        print("State: ",s,"  Value: ", V[s])
        
iterate()

State:  W   Value:  0
C
0
0
W
0
0
W
0
0
W
0
0
State:  N   Value:  0
C
0
0
N
0
0
N
0
0
N
0
0
State:  E   Value:  0
C
0
0
E
0
0
E
0
0
E
0
0
E
0
0
State:  S   Value:  0
C
0
0
S
0
0
S
0
0
S
0
0
State:  C   Value:  0
N
0
0
S
0
0
W
0
0
E
0
0
C
0
0
C
0
0
C
0
0
C
0
0
State:  W   Value:  0
State:  N   Value:  0
State:  E   Value:  0
State:  S   Value:  0
State:  C   Value:  0
